In [1]:

# contains folders locations and all
from setup import *

from cells_detection.cell import *
from cells_detection.cellConfig import *
from cells_detection.detection_model import *
from cells_detection.train_model import *
from itertools import product
from mrcnn import model as modellib
from mrcnn import visualize


Root directory /users/nfs/Etu7/21113797/PRAT/mrcnn


In [2]:
from tensorboard import notebook
notebook.list()

Known TensorBoard instances:
  - port 8000: logdir {TRAIN_SAVE_DIR} (started 11 days, 11:01:10 ago; pid 1326985)
  - port 6006: logdir /users/nfs/Etu7/21113797/train_results (started 11 days, 22:56:13 ago; pid 1240085)
  - port 8000: logdir /users/nfs/Etu7/21113797/PRAT/train_results (started 11 days, 12:05:17 ago; pid 1273188)


In [3]:
%load_ext tensorboard 
%tensorboard --logdir {TRAIN_SAVE_DIR} --host localhost --port 8080

In [4]:

# ############################################################
# #  TRAINING PHASE
# ############################################################
# weights = "coco"

# # weights = "usiigaci"

# print("Weights: ", weights)
# print("Dataset: ", DATASET_DIR)

# # Configurations

# config = CellConfig()
# config.display()

# # Create model

# model = modellib.MaskRCNN(mode="training", config=config,
#                                 model_dir=TRAIN_SAVE_DIR)


# selectWeights(model, weights="coco")

# # Train 
# train(model, DATASET_DIR, config)

In [5]:
############################################################
#  TRAINING PHASE
############################################################
weights = "coco"

# weights = "usiigaci"

print("Weights: ", weights)
print("Dataset: ", DATASET_DIR)

# Configurations

config = CellConfig()
config.display()

# Create model

model = modellib.MaskRCNN(mode="training", config=config,
                                model_dir=TRAIN_SAVE_DIR)


model = selectWeights(model, weights="coco")

# Train 
train(model, DATASET_DIR, config)

Weights:  coco
Dataset:  /users/nfs/Etu7/21113797/PRAT/Images

Configurations:
AUGMENT_TRAIN                  True
BACKBONE                       resnet50
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     6
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        400
DETECTION_MIN_CONFIDENCE       0.5
DETECTION_NMS_THRESHOLD        0.3
EPOCHS                         5
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 6
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  512
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  512
IMAGE_MIN_SCALE                2.0
IMAGE_RESIZE_MODE              crop
IMAGE_SHAPE                    [512 512   3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 

In [ ]:
parameters = dict(
    epochs = [20,40],
    # whether to augment training set or not
    is_aug = [True,False],
    lr  = [1e-3,1e-2,1e-1],
    training_method = ["all","head_all"],
    # STEPS PER EPOCH === BATCH SIZE is it better to set it according to dataset ? we'll know it asap
    bsize = [CellConfig().STEPS_PER_EPOCH,16,32],

    init_weights = ["coco"] # for the moment let s try with coco 
    # to consider someday
    # backbone = ["resnet50", "resnet101"]
)


param_values = [ v for v in parameters.values()]

for (epochs,is_aug,lr,train_mode,bsize,init_weights) in product(*param_values):

    print("Weights: ", init_weights)
    print("Dataset: ", DATASET_DIR)

    # Configurations

    config = CellConfig()

    # change the necessary params

    config.EPOCHS = epochs
    config.AUGMENT_TRAIN = is_aug
    config.STEPS_PER_EPOCH = bsize

    config.display()

    # selecting training mode

    modes = train_mode.split('_') # it always return a list even if there s no split carefull maam

    if(len(modes)==1):

        config.TRAINABLE_LAYERS = modes[0]

        # Create model
        model = modellib.MaskRCNN(mode="training", config=config,
                                        model_dir=TRAIN_SAVE_DIR)
        # select the weights 
        model = selectWeights(model, weights= init_weights)
        # Train 
        train(model, DATASET_DIR, config)
    
    else: # aka it s equal to 2

        #### first train the head
        config.TRAINABLE_LAYERS = modes[0]
        config.EPOCHS = epochs*0.35

        # Create model
        model = modellib.MaskRCNN(mode="training", config=config,
                                        model_dir=TRAIN_SAVE_DIR)
        # select the weights 
        model = selectWeights(model, weights= init_weights)
        # Train 
        train(model, DATASET_DIR, config)


        #### then train everything

        config.TRAINABLE_LAYERS = modes[1]
        # train the remaining epocs
        config.EPOCHS = epochs - config.EPOCHS
        
        # Create model
        model = modellib.MaskRCNN(mode="training", config=config,
                                        model_dir=TRAIN_SAVE_DIR)
        # select last saved weights 
        model = selectWeights(model, weights= "last")
        # Train 
        train(model, DATASET_DIR, config)

    

In [ ]:
############################################################
#  TEST PHASE 
############################################################

# weights = "last"

# phase = "inference"

# print("Weights: ", weights)
# print("Dataset: ", DATASET_DIR)


# # Configurations

# config = CellInferenceConfig()
# config.display()

# # Create model
# model = modellib.MaskRCNN(mode="inference", config=config,
#                                 model_dir=TRAIN_SAVE_DIR)

# # Select weights file to load
# if weights.lower() == "last":
#     # Find last trained weights
#     weights_path = model.find_last()

# elif weights.lower() == "imagenet":
#     # Start from ImageNet trained weights
#     weights_path = model.get_imagenet_weights()

# # Load weights
# print("Loading weights ", weights_path)
# model.load_weights(weights_path, by_name=True)

# # evaluate

# detect(model, DATASET_DIR, "test")
